# Version 2.5: 웨이팅 핫플레이스 요약문 생성 (배치 모드)

이 노트북은 **리뷰 데이터를 사용하지 않고**, 외부 정보만으로 웨이팅 핫플레이스의 요약문을 **일괄 생성**합니다.

## 📌 핵심 특징

### 데이터 소스 (5가지 소스 타입 기반)
- 🍔 **signature_menu** (시그니처 메뉴)
  - 대표 메뉴명과 비주얼 임팩트
  - 독특한 조합이나 퍼포먼스
  - SNS에서 화제가 된 이유
  
- 🎨 **atmosphere** (분위기)
  - 인테리어 컨셉 및 공간 특징
  - 포토존 위치 및 감성 키워드
  
- 🔥 **popularity** (인기도/웨이팅)
  - 평균 웨이팅 시간 (주말/평일)
  - SNS 바이럴 및 미디어 노출
  - 타겟 고객층
  
- 💵 **price_value** (가격/가치)
  - 대표 메뉴 가격대
  - 웨이팅 대비 경험 가치
  
- 📍 **location_access** (위치/접근성)
  - 지역명 및 지역 감성
  - 교통 및 주차 정보

### 프롬프트 아키텍처
- **시스템/유저 프롬프트 분리 패턴** 사용
- **Few-shot Learning**: 10개의 고품질 예시로 학습
- **JSON 출력 강제**: 구조화된 결과 보장

### 출력 형식
```json
{
  "shop_seq": 1,
  "shop_name": "매장명",
  "title": "SNS 핫플 성수동에서 크림 폭포 크로플로 감탄하는 디저트 카페",
  "summaries": [
    "화제성 + 웨이팅 정보 문장 (35-55자)",
    "시그니처 메뉴 비주얼 + 경험 문장 (35-55자)",
    "공간 감성 + 포토존 + 대기 가치 문장 (35-55자)"
  ]
}
```

## 🚀 사용 워크플로우

```
1. 섹션 1-3: 환경 설정 (최초 1회)
2. 섹션 4: SHOPS 리스트에 매장 정보 입력
3. 섹션 5: 건너뛰기 (섹션 4에서 정보 입력 완료)
4. 섹션 6: 프롬프트 실행 → 여러 매장 일괄 생성
5. 섹션 7: 결과 검증 (선택)
6. 섹션 8: JSON + CSV 파일 저장 (선택)
```

## ⚠️ 섹션 1-3: 최초 1회만 실행 (수정 금지)

In [ ]:
# 섹션 1: Package 설치
%pip install google-cloud-aiplatform google-genai python-dotenv

In [ ]:
# 섹션 2: 라이브러리 Import
import os
import json
import pandas as pd
import vertexai
from google import genai
from google.genai import types
from dotenv import load_dotenv

print("✅ 라이브러리 Import 완료")

In [ ]:
# 섹션 3: Vertex AI 초기화
# 환경 변수 로드
load_dotenv()

# GCP 프로젝트 설정
PROJECT_ID = "wad-dw"  # 실제 GCP 프로젝트 ID (프로젝트 이름 아님!)
LOCATION = "us-central1"

# Vertex AI 초기화
vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Vertex AI 초기화 완료: {PROJECT_ID} ({LOCATION})")

## ✅ 섹션 4: 매장 정보 입력 (수정 가능)

### 📋 입력 방법

1. **SHOPS 리스트**: 매장 기본 정보 (shop_seq, shop_name)
2. **COLLECTED_INFO 딕셔너리**: 매장별 수집된 정보를 매장명을 키로 입력

In [ ]:
# ========================================
# 매장 정보 입력
# ========================================

# 1️⃣ 매장 기본 정보 (shop_seq, shop_name)
SHOPS = [
    (1, "달맞이 광장 바베큐"),
    (2, "이재모피자 (본점)"),
    (3, "자매국수"),
    (4, "하이디라오 (명동점)"),
    (5, "산청숯불가든 (을지로)"),
    (6, "숙성도 (중문점)"),
    (7, "톤쇼우 (광안점)"),
    (8, "연돈"),
    (9, "카와카츠 (합정점)"),
    (10, "고기리막국수"),
]

# 2️⃣ 매장별 수집된 정보 (매장명을 키로 사용)
COLLECTED_INFO = {
    "달맞이 광장 바베큐": "",
    "이재모피자 (본점)": "",
    "자매국수": "",
    "하이디라오 (명동점)": "",
    "산청숯불가든 (을지로)": "",
    "숙성도 (중문점)": "",
    "톤쇼우 (광안점)": "",
    "연돈": "",
    "카와카츠 (합정점)": "",
    "고기리막국수": "",
}

# 3️⃣ SHOP_LIST 자동 생성 (shop_seq, shop_name, collected_info 매핑)
SHOP_LIST = [
    {
        "shop_seq": seq,
        "shop_name": name,
        "collected_info": COLLECTED_INFO.get(name, "[정보 없음 - COLLECTED_INFO에 추가 필요]")
    }
    for seq, name in SHOPS
]

# 출력
print(f"📍 다중 매장 배치 모드")
print(f"   - 총 매장 수: {len(SHOP_LIST)}개\n")
print("=" * 80)
for idx, shop in enumerate(SHOP_LIST, 1):
    info_length = len(shop['collected_info'].strip())
    status = "✅" if info_length > 100 else "⚠️"
    print(f"{status} [{idx}] {shop['shop_name']} (seq: {shop['shop_seq']}) - {info_length:,}자")
print("=" * 80)

if any(len(shop['collected_info'].strip()) < 100 for shop in SHOP_LIST):
    print("\n⚠️ 일부 매장의 정보가 부족합니다. COLLECTED_INFO를 확인하세요.")

## ✅ 섹션 5: 소스 타입별 정보 구조 안내

### 📂 웨이팅 핫플레이스 소스 타입 (5가지)

섹션 4의 `COLLECTED_INFO`에 입력할 정보는 아래 5가지 소스 타입을 포함해야 합니다:

#### 1️⃣ signature_menu (시그니처 메뉴)
- 대표 메뉴명과 특징
- 비주얼 임팩트 (높이, 크기, 색감)
- 독특한 조합이나 퍼포먼스
- 맛과 식감 설명
- SNS에서 화제가 된 이유

#### 2️⃣ atmosphere (분위기)
- 인테리어 컨셉 (미니멀, 빈티지, 모던 등)
- 공간 특징 (통창, 야외석, 오픈 키치)
- 조명과 색감
- 포토존 위치
- 감성 키워드 (힙함, 세련됨, 아늑함)

#### 3️⃣ popularity (인기도/웨이팅)
- 평균 웨이팅 시간 (주말/평일)
- 피크 타임
- SNS 바이럴 여부
- 연예인 방문 이력
- 미디어 노출 (TV, 유튜브, 인스타그램)
- 입소문 확산 경로

#### 4️⃣ price_value (가격/가치)
- 대표 메뉴 가격대
- 1인당 평균 비용
- 웨이팅 대비 가치
- 특별한 경험의 가치
- **주의**: 웨이팅 핫플은 "가성비"보다 "경험 가치" 강조

#### 5️⃣ location_access (위치/접근성)
- 지역명 (성수동, 망원동, 을지로 등)
- 지역 감성 (힙함, 트렌디함)
- 교통 (지하철역 거리)
- 주차 가능 여부
- 찾기 쉬운지/숨은 맛집인지

### 💡 정보 입력 예시

```python
COLLECTED_INFO = {
    "달맞이 광장 바베큐": """
    ## SIGNATURE_MENU
    - 메뉴명: 크림 크로플
    - 비주얼: 높이 20cm로 쌓인 생크림이 폭포처럼 흘러내림
    - 특징: 바삭한 와플과 부드러운 크림의 조화
    - 화제성: 인스타그램에서 10만 개 이상 해시태그
    
    ## ATMOSPHERE
    - 인테리어: 화이트톤 미니멀 디자인
    - 공간 특징: 대형 통창으로 자연광 유입
    - 포토존: 창가 자리, 통창 배경
    - 감성: 인스타그램 감성, 브런치 카페 무드
    
    ## POPULARITY
    - 웨이팅: 주말 2시간, 평일 1시간
    - 피크 타임: 주말 오후 1-3시
    - SNS: 인스타그램 바이럴, 크림 폭포로 화제
    - 타겟: MZ세대, 20-30대 여성
    
    ## PRICE_VALUE
    - 시그니처 가격: 크림 크로플 18,000원
    - 1인 평균: 20,000-25,000원
    - 경험 가치: 웨이팅해도 갈 만한 비주얼과 맛
    
    ## LOCATION_ACCESS
    - 지역: 성수동
    - 감성: 힙한 성수동 감성
    - 교통: 지하철 2호선 성수역 도보 5분
    - 주차: 인근 공영주차장 이용
    """
}
```

⚠️ **주의**: 이 섹션은 정보 구조를 안내하는 용도입니다. 실제 정보는 섹션 4의 `COLLECTED_INFO`에 입력하세요.

In [ ]:
# ========================================
# 정보 입력 확인
# ========================================

print("📥 매장별 정보 입력 현황\n")
print("=" * 80)

for idx, shop in enumerate(SHOP_LIST, 1):
    shop_name = shop['shop_name']
    collected_info = shop['collected_info']
    info_length = len(collected_info.strip())
    
    # 상태 표시
    if info_length < 50:
        status = "❌ 정보 없음"
    elif info_length < 200:
        status = "⚠️ 정보 부족"
    else:
        status = "✅ 정보 충분"
    
    print(f"[{idx}] {shop_name}")
    print(f"    상태: {status} ({info_length:,}자)")
    print(f"    권장: 각 소스 타입당 100~300자 (총 500~1500자)")
    print()

print("=" * 80)
print("\n⚠️ 정보가 부족한 매장은 섹션 4의 COLLECTED_INFO를 수정하세요.")
print("➡️ 정보 입력이 완료되었으면 섹션 6을 실행하세요.")

## ✅ 섹션 6: 프롬프트 작성 및 요약문 생성 (수정 가능)

### 프롬프트 구조 설계

이 노트북은 **시스템/유저 프롬프트 분리 패턴**을 사용합니다:

#### 📌 시스템 프롬프트 (System Prompt)
- **역할**: AI의 정체성, 규칙, 출력 형식 정의
- **특징**: 고정된 내용 (매장마다 동일)
- **내용**: 웨이팅 핫플 작성 원칙, 구조 규칙, 출력 형식

#### 📌 유저 프롬프트 (User Prompt)
- **역할**: 구체적인 작업 요청 및 데이터 제공
- **특징**: 매장마다 변경되는 내용
- **내용**: 매장 정보, 수집된 정보, Few-shot 예시

#### 🔗 프롬프트 결합
Google Gemini는 시스템/유저를 API 레벨에서 구분하지 않으므로, 전송 시 하나의 문자열로 결합합니다.

### 프롬프트 작성 규칙
- ❌ **f-string 사용 금지** (변수가 아직 정의되지 않아 에러)
- ✅ **일반 문자열 + `.format()` 사용**
- ✅ **JSON 예시에서 `{{` `}}` 사용** (이스케이프)

In [ ]:
# ========================================
# 프롬프트 정의
# ========================================

# -----------------------------------------
# [시스템 프롬프트] AI의 정체성과 규칙
# -----------------------------------------
system_prompt = """
당신은 웨이팅 핫플레이스 전문 작가입니다.
리뷰 데이터 없이 외부 정보만으로 "왜 줄 서서라도 가야 하는가?"를 명확히 전달하는 매장 요약문을 작성합니다.

## 작성 원칙
1. **화제성**: SNS 바이럴, 입소문, 미디어 노출을 강조
2. **비주얼**: 인스타그래머블, 포토제닉한 요소 부각
3. **경험 가치**: 웨이팅해도 갈 만한 특별한 경험 전달
4. **생동감**: 흥미롭고 트렌디한 어조 사용
5. **대기 정당화**: 웨이팅 시간과 대기 가치를 명확히 제시
6. **독창성**: 예시를 그대로 복사하지 말고, 톤앤매너만 참고하여 해당 매장만의 고유한 특징을 작성

## 소스 타입별 정보 활용 가이드

[수집된 정보]에는 5가지 소스 타입이 포함되어 있습니다. 각 소스 타입을 다음과 같이 활용하세요:

### 1️⃣ SIGNATURE_MENU (시그니처 메뉴)
- **활용 위치**: 주로 **2번 문장**(시그니처 메뉴 비주얼 + 맛/식감)과 **제목**에 활용
- **내용**: 대표 메뉴명, 비주얼 임팩트, 독특한 조합, 맛과 식감
- **예시**: "크림 크로플은 높이 20cm로 쌓인 생크림이 폭포처럼 흘러내리는 비주얼"

### 2️⃣ ATMOSPHERE (분위기)
- **활용 위치**: 주로 **3번 문장**(공간 감성 + 포토존)에 활용
- **내용**: 인테리어 컨셉, 공간 특징, 포토존, 감성 키워드
- **예시**: "화이트톤 미니멀 인테리어와 대형 통창이 어우러진 공간"

### 3️⃣ POPULARITY (인기도/웨이팅)
- **활용 위치**: 주로 **1번 문장**(화제성 + 웨이팅 정보)과 **제목**에 활용
- **내용**: 웨이팅 시간, SNS 바이럴, 미디어 노출, 타겟 고객층
- **예시**: "MZ세대 사이에서 SNS 바이럴로 주말 2시간 웨이팅을 기록"

### 4️⃣ PRICE_VALUE (가격/가치)
- **활용 위치**: 필요시 **1번 또는 3번 문장**에 보조 정보로 활용
- **내용**: 가격대보다는 "웨이팅 대비 경험 가치" 강조
- **주의**: "가성비" 표현 대신 "특별한 경험", "대기 가치" 등으로 표현

### 5️⃣ LOCATION_ACCESS (위치/접근성)
- **활용 위치**: 주로 **1번 문장**(지역 + 지역 감성)에 활용
- **내용**: 지역명, 지역 감성 (성수동 힙함, 망원동 핫플 등)
- **예시**: "성수동에서", "입소문 난 망원동"

## 요약 문장 구조
- **1번 문장**: [지역/컨셉] + [화제성 키워드] + [웨이팅 정보] + [타겟 고객층]
  → 주로 LOCATION_ACCESS + POPULARITY 활용
  
- **2번 문장**: [시그니처 메뉴 비주얼] + [독특한 경험] + [맛/식감]
  → 주로 SIGNATURE_MENU 활용
  
- **3번 문장**: [공간 감성] + [포토존] + [방문 용도] + [대기 가치]
  → 주로 ATMOSPHERE 활용, 필요시 PRICE_VALUE(경험 가치)

## 제목 작성 규칙
- 형식: "[화제성 키워드]에서, [시그니처 비주얼 메뉴]로 [감정 동사]하는 [컨셉]"
- 예시: "SNS 핫플 성수동에서, 크림 폭포 크로플로 감탄하는 디저트 카페"
- 길이: 20-40자
- 화제성 키워드 필수 포함
- 주로 LOCATION_ACCESS + SIGNATURE_MENU + POPULARITY 활용

## 웨이팅 핫플 특화 가이드
- ✅ 웨이팅 정보 명시 (주말 2시간, 평일 1시간 등)
- ✅ 화제성 강조 (SNS 바이럴, 입소문, 연예인 단골)
- ✅ 비주얼 임팩트 (높이 쌓인, 폭포처럼, 불쇼)
- ✅ 타겟 고객층 (MZ세대, 20-30대 여성, 커플)
- ✅ 감성 표현 (감탄, 놀라움, 매료, 황홀)
- ✅ 포토존 언급 (인스타 성지, 창가 자리)
- ✅ 경험 가치 (색다른, 특별한, 이색적)
- ❌ 가성비/실용성 표현 금지
- ❌ 조용한/차분한 표현 금지

## 출력 형식
반드시 유효한 JSON 형식으로만 출력하세요. 다른 설명이나 주석은 포함하지 마세요.
{{
  "shop_seq": 숫자,
  "shop_name": "매장명",
  "title": "제목",
  "summaries": ["문장1", "문장2", "문장3"]
}}
"""

# -----------------------------------------
# [유저 프롬프트 템플릿] 구체적 작업 내용
# -----------------------------------------
user_prompt_template = """
[매장 정보]
매장명: {shop_name}
Shop Seq: {shop_seq}

[수집된 정보]
{collected_info}

[참고 예시 - 아래 예시들의 톤앤매너와 문체만 참고하세요. 내용은 절대 복사하지 마세요.]

**중요**: 아래 예시는 작성 스타일을 보여주기 위한 참고용입니다. 예시의 내용을 그대로 사용하지 말고, 
수집된 정보를 바탕으로 해당 매장만의 고유한 특징을 작성하세요.

---

**예시 1: 달맞이 광장 바베큐**
{{
  "title": "을지로 핫플, 숯불 불향 가득한 야외 바비큐로 즐기는 레트로 감성 맛집",
  "summaries": [
    "달맞이 광장 바베큐는 야외 테이블에서 재래식 숯불 바베큐와 생맥주를 즐기는 레트로 감성 맛집입니다.",
    "시그니처인 소금 바베큐는 닭다리 부위에 진한 숯불 향을 입혀내며, 신선한 생맥주, 생감자튀김과 함께 주문하는 것을 추천합니다.",
    "특히 야외 테이블과 생동감 넘치고 활기찬 분위기로 큰 인기를 끌고 있는 을지로의 핫플입니다. 레트로 감성의 야외 분위기 덕분에 퇴근 후 모임 장소로 큰 인기를 끌고 있습니다. 을지로 3가역에서 도보 1분 거리에 위치합니다."
  ]
}}

**예시 2: 이재모피자 (본점)**
{{
  "title": "부산 서면, 치즈 크러스트 가득 담백하고 고소한 피자로 만족하는 상징적 맛집",
  "summaries": [
    "이재모피자 본점은 1992년부터 시작된 서면역 인근 부산의 상징적 피자 맛집입니다.",
    "담백하고 고소한 모차렐라 치즈 크러스트 피자가 핵심이며, 본점 한정 김치볶음밥을 함께 곁들이는 조합을 추천합니다. 2~3만원대의 가격대로 가족 단위 방문객에게 인기가 높습니다.",
    "중앙역에서 도보 8분 거리에 위치합니다. 포장 주문 시, 웨이팅 없이 이용할 수 있습니다."
  ]
}}

**예시 3: 자매국수**
{{
  "title": "제주 시내, 진한 고기 육수의 제주고기국수 원조로 만족하는 전통 맛집",
  "summaries": [
    "자매국수는 진하고 깊은 돼지 사골 육수를 사용하는 제주 고기국수 원조 맛집입니다.",
    "시그니처인 고기국수는 뽀얀 육수와 부드러운 돼지고기가 조화를 이루며, 비빔국수와 돔베고기를 함께 주문하는 조합이 인기입니다.",
    "1만원대로 합리적이며, 제주 여행 중 든든한 식사를 위해 웨이팅을 감수하고 방문할만한 곳입니다. 제주 시내에 위치하며 매장 내 주차장이 있어 주차가 편리합니다."
  ]
}}

**예시 4: 하이디라오 (명동점)**
{{
  "title": "명동, 다양한 탕 베이스와 세심한 서비스의 중국식 훠궈로 만족하는 글로벌 전문점",
  "summaries": [
    "하이디라오 명동점은 다양한 탕 베이스와 세심한 서비스를 앞세운 글로벌 훠궈 전문점입니다.",
    "4컷 탕 시스템을 활용해 청유마라, 토마토 등 취향에 맞는 여러 맛을 즐길 수 있습니다. 특히 토마토탕 베이스와 새우 완자가 추천되며, DIY 소스바가 이용의 재미를 더합니다.",
    "네일 아트, 간식 제공 등 독특한 서비스로 긍정적인 입소문이 자자하며 만족도가 높습니다. 명동역에서 도보 3분 거리에 위치합니다."
  ]
}}

**예시 5: 산청숯불가든 (을지로)**
{{
  "title": "을지로 레트로, 옛 공간의 감성 곁들인 숯불 흑돼지 소금구이로 즐기는 고깃집",
  "summaries": [
    "산청숯불가든은 재래식 숯불에 지리산 흑돼지 소금구이를 구워내는 레트로 감성의 고깃집입니다. 직원이 구워주는 서비스로 높은 만족도를 얻고 있으며, 을지로 노포 감성을 현대적으로 재해석하여 큰 인기를 끌고 있습니다.",
    "시그니처인 숯불 향의 흑돼지 소금구이는 쫄깃한 육질이 특징이며, 지리산 칡냉면을 곁들이는 조합이 추천됩니다. 2만원대 초중반의 가격대이며, 활기찬 모임 분위기가 특징입니다.",
    "을지로3가역에서 도보 1분 거리에 위치합니다."
  ]
}}

**예시 6: 숙성도 (중문점)**
{{
  "title": "제주 중문, 압도적 숙성 기술이 돋보이는 흑돼지 뼈등심으로 감탄하는 대표 맛집",
  "summaries": [
    "숙성도는 720시간 이상 숙성된 프리미엄 제주 흑돼지를 전문가가 구워주는 식사 전문점입니다. 장시간 숙성된 고기의 풍부한 육즙에 대한 대중의 극찬이 이어지며 제주 대표 맛집으로 인기가 매우 높습니다.",
    "시그니처인 960 흑뼈등심은 한정 판매되며, 고기를 먹은 뒤 갈치속젓 볶음밥과의 조합이 추천됩니다.",
    "고급스럽고 깔끔한 분위기에서 식사를 즐길 수 있으며, 중문단지 인근에 위치합니다. 원격 줄서기를 활용하는 것이 팁입니다."
  ]
}}

**예시 7: 톤쇼우 (광안점)**
{{
  "title": "광안리, 부드러운 안심과 지방의 풍미가 조화로운 돈카츠로 만족하는 유명 맛집",
  "summaries": [
    "톤쇼우는 습식 숙성된 최상급 돼지고기로 겉바속촉 돈카츠를 내세우는 유명 유튜버 다수에게 소개된 유명한 맛집입니다. 부드러운 식감과 풍부한 육즙에 대한 많은 이들의 호평이 이어지는 곳입니다.",
    "습식 숙성된 특로스카츠와 부드러운 히레카츠가 시그니처이며, 말돈 소금 또는 와사비와 조합하는 것이 추천됩니다.",
    "광안리 해변 근처라 뷰가 좋고, 혼밥에 적합합니다. 광안역에서 도보 12분 거리에 위치합니다."
  ]
}}

**예시 8: 연돈**
{{
  "title": "제주 서귀포, 치즈가 폭포수처럼 흐르는 치즈카츠로 매료되는 최고 인기 맛집",
  "summaries": [
    "연돈은 바삭한 튀김옷과 폭포수 치즈가 특징인 수제 돈카츠를 판매하는 최고의 인기 맛집입니다. 미디어 출연 후 전국적인 신드롬을 일으킨 최고의 명소입니다.",
    "1만원대 초반의 훌륭한 가성비에 대해 대다수의 지지를 얻고 있으며, 커팅 시 치즈가 폭포수처럼 흐르는 치즈카츠가 시그니처입니다.",
    "제주 서귀포시에 위치하며, 현재 현장 웨이팅만 가능해 오픈런이 필수입니다."
  ]
}}

**예시 9: 카와카츠 (합정점)**
{{
  "title": "합정, 파동 숙성 기술의 겉바속촉 프리미엄 카츠로 감탄하는 돈카츠 전문점",
  "summaries": [
    "카와카츠는 파동 숙성 기술을 적용한 히레카츠와 특로스카츠를 선보이는 프리미엄 돈카츠 전문점입니다. 열흘간의 파동 숙성을 통해 부드러운 육질을 극대화하는 차별화된 조리법으로 미식가들의 호평이 이어집니다.",
    "시그니처인 특로스카츠와 히레카츠는 말돈 소금과 와사비를 곁들여 즐기는 것이 추천됩니다.",
    "합정역에서 도보 3분 거리에 위치합니다. 바 좌석 위주라 혼밥에 좋습니다."
  ]
}}

**예시 10: 고기리막국수**
{{
  "title": "들기름 막국수 원조, 들기름의 고소함과 메밀 면의 조화로 만족하는 전통 국수 전문점",
  "summaries": [
    "고기리막국수는 2012년 최초로 들기름막국수를 시작하여 이 메뉴를 대중화시킨 들기름 막국수의 원조격인 전통 국수 전문점입니다. 순도 높은 들기름의 고소함이 많은 이들의 호평을 받으며 들기름 막국수 열풍을 일으켰습니다.",
    "시그니처인 들기름 막국수는 순도 높은 들기름의 고소함과 메밀 면의 조화가 특징이며, 부드러운 수육과 함께 즐기고 나중에 동치미 육수를 부어 먹는 조합이 추천됩니다.",
    "매장 내부는 한옥 분위기로 깔끔하며, 원격 줄서기를 활용할 수 있습니다."
  ]
}}

---

**다시 한 번 강조**: 위 예시는 작성 스타일(톤, 문체, 구조)을 참고하기 위한 것입니다.
예시의 구체적인 내용(메뉴명, 지역명, 특징 등)을 그대로 사용하지 말고,
[수집된 정보]를 바탕으로 [{shop_name}] 매장만의 고유한 특징을 작성하세요.
"""

print("✅ 프롬프트 템플릿 작성 완료")
print("   - 시스템 프롬프트: 웨이팅 핫플 작성 원칙 + 소스 타입 활용 가이드")
print("   - 유저 프롬프트: 구체적 작업 내용")
print("   - Few-shot 예시: 10개 (톤앤매너 참고용)")

In [ ]:
# ========================================
# 모델 설정 및 LLM 호출
# ========================================

# -----------------------------------------
# 모델 파라미터 설정
# -----------------------------------------
MODEL_NAME = "gemini-2.5-pro"  # 높은 품질
TEMPERATURE = 0.6  # 중저가보다 약간 높은 창의성
MAX_OUTPUT_TOKENS = 4096  # JSON 응답을 위한 충분한 토큰

generation_config = types.GenerateContentConfig(
    temperature=TEMPERATURE,
    max_output_tokens=MAX_OUTPUT_TOKENS,
    response_mime_type="application/json"  # JSON 출력 강제
)

print(f"🤖 모델: {MODEL_NAME}")
print(f"   - Temperature: {TEMPERATURE}")
print(f"   - Max Output Tokens: {MAX_OUTPUT_TOKENS:,}\n")

# -----------------------------------------
# 요약문 생성 함수
# -----------------------------------------
def generate_summary(shop_seq, shop_name, collected_info):
    """단일 매장에 대한 요약문 생성"""
    
    # 유저 프롬프트 생성
    user_prompt = user_prompt_template.format(
        shop_seq=shop_seq,
        shop_name=shop_name,
        collected_info=collected_info
    )
    
    # 프롬프트 결합
    combined_prompt = f"{system_prompt}\n\n{user_prompt}"
    
    try:
        # LLM 호출
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=combined_prompt,
            config=generation_config
        )
        
        # JSON 파싱
        result_text = response.text
        result_json = json.loads(result_text)
        
        # 토큰 사용량 추출
        usage_metadata = response.usage_metadata if hasattr(response, 'usage_metadata') else None
        token_info = {}
        if usage_metadata:
            token_info = {
                "input_tokens": usage_metadata.prompt_token_count,
                "output_tokens": usage_metadata.candidates_token_count,
                "total_tokens": usage_metadata.total_token_count
            }
        
        return {
            "status": "success",
            "data": result_json,
            "tokens": token_info
        }
        
    except json.JSONDecodeError as e:
        return {
            "status": "error",
            "error_type": "JSONDecodeError",
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {}
        }
        
    except Exception as e:
        return {
            "status": "error",
            "error_type": type(e).__name__,
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {}
        }

# -----------------------------------------
# 다중 매장 배치 처리
# -----------------------------------------
all_results = []
total_input_tokens = 0
total_output_tokens = 0
total_tokens = 0

print(f"📝 {len(SHOP_LIST)}개 매장 요약문 생성 시작\n")
print("=" * 80)

for idx, shop in enumerate(SHOP_LIST, 1):
    shop_seq = shop["shop_seq"]
    shop_name = shop["shop_name"]
    collected_info = shop["collected_info"]
    
    print(f"\n[{idx}/{len(SHOP_LIST)}] {shop_name} 처리 중...")
    
    result = generate_summary(shop_seq, shop_name, collected_info)
    
    if result["status"] == "success":
        result_json = result["data"]
        all_results.append(result_json)
        
        # 토큰 사용량 출력 및 누적
        tokens = result.get("tokens", {})
        if tokens:
            input_tokens = tokens.get('input_tokens', 0)
            output_tokens = tokens.get('output_tokens', 0)
            total_tokens_shop = tokens.get('total_tokens', 0)
            
            total_input_tokens += input_tokens
            total_output_tokens += output_tokens
            total_tokens += total_tokens_shop
            
            print(f"   ✅ 성공 | Input: {input_tokens:,} | Output: {output_tokens:,} | Total: {total_tokens_shop:,} tokens")
        else:
            print(f"   ✅ 성공")
    else:
        print(f"   ❌ 실패: {result['error_type']} - {result['message']}")
        # 실패한 경우에도 기록
        all_results.append({
            "shop_seq": shop_seq,
            "shop_name": shop_name,
            "status": "error",
            "error": result['message']
        })

print("\n" + "=" * 80)
print(f"\n✅ 전체 완료: {len(all_results)}개 매장 처리")

# 성공/실패 개수
success_count = sum(1 for r in all_results if r.get("status") != "error")
error_count = len(all_results) - success_count
print(f"   - 성공: {success_count}개")
if error_count > 0:
    print(f"   - 실패: {error_count}개")

# 총 토큰 사용량 출력
if total_tokens > 0:
    print(f"\n📊 총 토큰 사용량:")
    print(f"   - Total Input:  {total_input_tokens:,} tokens")
    print(f"   - Total Output: {total_output_tokens:,} tokens")
    print(f"   - Total:        {total_tokens:,} tokens")

# -----------------------------------------
# 전체 결과 상세 출력
# -----------------------------------------
print("\n" + "=" * 80)
print("📋 전체 결과 상세보기")
print("=" * 80)

for idx, result in enumerate(all_results, 1):
    print(f"\n{'─' * 80}")
    print(f"[{idx}] {result.get('shop_name')}")
    print(f"{'─' * 80}")
    
    if result.get("status") != "error":
        # 성공한 경우 - 전체 내용 출력
        print(f"\n📌 제목:")
        print(f"   {result.get('title')}")
        
        print(f"\n📝 요약문:")
        summaries = result.get('summaries', [])
        for i, summary in enumerate(summaries, 1):
            print(f"   {i}. {summary}")
        
        print(f"\n✅ 상태: 성공")
    else:
        # 실패한 경우
        print(f"\n❌ 상태: 에러")
        print(f"   오류 내용: {result.get('error', 'N/A')}")

print("\n" + "=" * 80)

## ✅ 섹션 7: 후처리 검증 (선택 사항)

### 검증 항목

이 섹션은 생성된 요약문의 품질을 **코드 기반**으로 검증합니다 (LLM 호출 없음):

#### 📋 기본 구조 검증
- 필수 키 존재 확인 (shop_seq, shop_name, title, summaries)
- 요약문 개수 (정확히 3개)
- 문장 길이 (35-55자 권장, 30자 미만/80자 초과 경고)
- 제목 길이 (20-40자 권장)

#### 🚫 품질 검증
- 예시 복사 의심 (예시와 90% 이상 유사)
- 웨이팅 부적합 표현 (가성비, 조용한, 차분한)
- 문장 간 중복 내용 (3개 문장이 너무 유사)
- 특수문자/이모지 체크

#### 💡 검증 결과
- ✅ 통과: 모든 규칙 준수
- ⚠️ 경고: 일부 권장사항 위반 (사용 가능하지만 검토 권장)
- ❌ 실패: 필수 규칙 위반 (수정 필요)

In [ ]:
# ========================================
# 강력한 후처리 검증 함수
# ========================================

import difflib
import re

# -----------------------------------------
# 예시 문장 리스트 (복사 의심 체크용)
# -----------------------------------------
EXAMPLE_SENTENCES = [
    "성수동에서 크림이 폭포처럼 쏟아지는 크로플을 내세워 MZ세대 사이에서 SNS 바이럴로 주말 2시간 웨이팅을 기록하는 핫플레이스입니다.",
    "시그니처인 크림 크로플은 높이 20cm로 쌓인 생크림이 폭포처럼 흘러내리는 비주얼로 인스타그램을 점령하며, 바삭한 와플과 부드러운 크림의 조화가 중독성을 선사합니다.",
    "화이트톤 미니멀 인테리어와 대형 통창이 어우러진 공간은 자연광이 쏟아지는 창가 자리가 인기 포토존으로, 친구와의 브런치나 데이트 장소로 SNS 감성을 완성합니다.",
    "망원동에서 숯불 직화로 구운 연어덮밥을 내세워 20-30대 여성 사이에서 입소문으로 평일에도 1시간 웨이팅이 필수인 일식당입니다.",
    "시그니처인 숯불 연어덮밥은 눈앞에서 직화로 그을리는 퍼포먼스가 압권이며, 겉은 고소하고 속은 부드러운 연어의 식감이 감탄을 자아냅니다.",
]

# -----------------------------------------
# 부적합 키워드 리스트 (웨이팅 핫플에 맞지 않는 표현)
# -----------------------------------------
INAPPROPRIATE_KEYWORDS = [
    "가성비", "합리적", "저렴", "부담 없는",
    "조용한", "차분한", "격식 있는", "고급스러운",
    "실용적", "효율적", "편리한"
]

def calculate_similarity(text1, text2):
    """두 문자열 간 유사도 계산 (0.0 ~ 1.0)"""
    return difflib.SequenceMatcher(None, text1, text2).ratio()

def check_example_similarity(sentence):
    """예시 문장과의 유사도 체크"""
    max_similarity = 0.0
    most_similar_example = None
    
    for example in EXAMPLE_SENTENCES:
        similarity = calculate_similarity(sentence, example)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_example = example
    
    return max_similarity, most_similar_example

def check_inappropriate_keywords(text):
    """부적합 키워드 체크"""
    found_keywords = []
    for keyword in INAPPROPRIATE_KEYWORDS:
        if keyword in text:
            found_keywords.append(keyword)
    return found_keywords

def check_special_chars(text):
    """특수문자/이모지 체크"""
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "]+", 
        flags=re.UNICODE
    )
    
    emojis = emoji_pattern.findall(text)
    return emojis

def validate_summary(result, strict=False):
    errors = []
    warnings = []
    
    # 1. 필수 키 존재 확인
    required_keys = ["shop_seq", "shop_name", "title", "summaries"]
    for key in required_keys:
        if key not in result:
            errors.append(f"❌ 필수 키 누락: {key}")
    
    if errors:
        return {"passed": False, "errors": errors, "warnings": warnings}
    
    # 2. 요약문 개수 확인
    summaries = result.get("summaries", [])
    if len(summaries) != 3:
        errors.append(f"❌ 요약문 개수 오류: {len(summaries)}개 (3개 필요)")
        return {"passed": False, "errors": errors, "warnings": warnings}
    
    # 3. 문장 길이 확인 (웨이팅 핫플은 조금 더 짧고 임팩트 있게)
    for i, sentence in enumerate(summaries, 1):
        length = len(sentence)
        if length < 30:
            warnings.append(f"⚠️ {i}번 문장이 너무 짧음: {length}자 (권장 35-55자)")
        elif length > 80:
            warnings.append(f"⚠️ {i}번 문장이 너무 김: {length}자 (권장 35-55자)")
    
    # 4. 제목 길이 확인
    title = result.get("title", "")
    title_length = len(title)
    if title_length < 15:
        warnings.append(f"⚠️ 제목이 너무 짧음: {title_length}자 (권장 20-40자)")
    elif title_length > 50:
        warnings.append(f"⚠️ 제목이 너무 김: {title_length}자 (권장 20-40자)")
    
    # 5. 예시 복사 의심 체크
    for i, sentence in enumerate(summaries, 1):
        similarity, similar_example = check_example_similarity(sentence)
        if similarity >= 0.9:
            errors.append(f"❌ {i}번 문장이 예시를 복사한 것으로 의심됨 (유사도 {similarity:.1%})")
        elif similarity >= 0.7:
            warnings.append(f"⚠️ {i}번 문장이 예시와 유사함 (유사도 {similarity:.1%})")
    
    # 6. 부적합 키워드 체크
    all_text = title + " ".join(summaries)
    inappropriate_found = check_inappropriate_keywords(all_text)
    if inappropriate_found:
        warnings.append(f"⚠️ 웨이팅 핫플에 부적합한 표현: {', '.join(inappropriate_found)}")
    
    # 7. 특수문자/이모지 체크
    emojis = check_special_chars(all_text)
    if emojis:
        warnings.append(f"⚠️ 이모지 발견: {', '.join(emojis)}")
    
    # 8. 문장 간 중복 체크
    for i in range(len(summaries)):
        for j in range(i+1, len(summaries)):
            similarity = calculate_similarity(summaries[i], summaries[j])
            if similarity >= 0.7:
                warnings.append(f"⚠️ {i+1}번과 {j+1}번 문장이 너무 유사함 (유사도 {similarity:.1%})")
    
    # 결과 판정
    if strict:
        passed = len(errors) == 0 and len(warnings) == 0
    else:
        passed = len(errors) == 0
    
    return {"passed": passed, "errors": errors, "warnings": warnings}

# 전체 결과 검증 실행
if len(all_results) == 0:
    print("⚠️ 검증할 결과가 없습니다.")
else:
    print("🔍 요약문 검증 시작\n")
    print("=" * 80)
    
    total_passed = 0
    total_warnings = 0
    total_failed = 0
    
    for idx, result in enumerate(all_results, 1):
        if result.get("status") == "error":
            continue
        
        shop_name = result.get("shop_name", "Unknown")
        print(f"\n[{idx}] {shop_name}")
        print("─" * 80)
        
        validation = validate_summary(result, strict=False)
        errors = validation["errors"]
        warnings = validation["warnings"]
        passed = validation["passed"]
        
        if passed and len(warnings) == 0:
            print("✅ 검증 통과: 모든 규칙을 준수합니다")
            total_passed += 1
        elif passed:
            print(f"✅ 검증 통과 (경고 {len(warnings)}개)")
            total_passed += 1
            total_warnings += 1
            for warning in warnings:
                print(f"   {warning}")
        else:
            print(f"❌ 검증 실패 (오류 {len(errors)}개)")
            total_failed += 1
            for error in errors:
                print(f"   {error}")
    
    print("\n" + "=" * 80)
    print("📊 검증 요약")
    print("=" * 80)
    print(f"✅ 통과: {total_passed}개")
    if total_warnings > 0:
        print(f"⚠️ 경고 있음: {total_warnings}개")
    if total_failed > 0:
        print(f"❌ 실패: {total_failed}개")

## ✅ 섹션 8: 결과 저장 (선택 사항)

### 저장 형식

#### 단일 매장 모드
- JSON 파일 1개
- 파일명: `summary_{매장명}_{타임스탬프}.json`

#### 다중 매장 모드
- JSON 파일 + CSV 파일
- 파일명: `summaries_batch_{타임스탬프}.json/csv`

In [ ]:
# ========================================
# 결과 저장
# ========================================

from datetime import datetime
import pandas as pd

if len(all_results) == 0:
    print("⚠️ 저장할 결과가 없습니다.")
else:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # JSON 파일
    json_filename = f"summaries_batch_{timestamp}.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 저장 완료: {json_filename}")
    
    # CSV 파일
    csv_data = []
    for result in all_results:
        if result.get("status") != "error":
            csv_data.append({
                "shop_seq": result.get("shop_seq"),
                "shop_name": result.get("shop_name"),
                "title": result.get("title"),
                "summary_1": result.get("summaries", [])[0] if len(result.get("summaries", [])) > 0 else "",
                "summary_2": result.get("summaries", [])[1] if len(result.get("summaries", [])) > 1 else "",
                "summary_3": result.get("summaries", [])[2] if len(result.get("summaries", [])) > 2 else "",
                "status": "success"
            })
    
    df = pd.DataFrame(csv_data)
    csv_filename = f"summaries_batch_{timestamp}.csv"
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    
    print(f"✅ CSV 저장 완료: {csv_filename}")